# How to build a simple Agent LLM App with LangGraph


In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

## Agents
* Agents use LLMs as reasoning engines to determine which actions to take.

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("Who are the top stars of the 2024 Eurocup?")

[{'url': 'https://talksport.com/football/1755223/euro-2024-best-players-cristiano-ronaldo-harry-kane-jude-bellingham-kylian-mbappe/',
  'content': "Kane is just one of the names of serious talent that are set to star at Euro 2024 Credit: Getty. Gareth Southgate's side face stiff competition though as World Cup finalists France remain one of ..."},
 {'url': 'https://www.nbcsports.com/soccer/news/euro-2024-player-power-rankings-who-are-the-top-20-stars',
  'content': 'Plus, every tournament there are always stars who emerge to make a name for themselves and players from Georgia and Austria are standing tall. Below are the top 20 players at EURO 2024 based on current form and we will update it throughout the tournament.'}]

In [8]:
tools = [search]

In [9]:
llm_with_tools = llm.bind_tools(tools)

### Create the agent¶

In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

### Run the agent¶

In [11]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})

response["messages"]

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', id='012dc39f-b951-4bd9-a04e-f9f3335a1a0f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_r56nPjRGP3u2SXowwCFzCp45', 'function': {'arguments': '{"query":"soccer Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 92, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-32e1f432-6a7d-49f7-b0e6-21e0e5f54e59-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'soccer Eurocup 2024 location'}, 'id': 'call_r56nPjRGP3u2SXowwCFzCp45', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 25, 'total_tokens': 117}),
 ToolMessage(content='[{"url": "https://apnews.com/article/euro-2024-germany-stadiums-9b6953fa87d168a754df5b2480b75010", "content": "F

In [12]:
## stream


for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="When and where will it be the 2024 Eurocup final match?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Gt5NplksLy9mAZXgoFxYdYxE', 'function': {'arguments': '{"query":"2024 Eurocup final match date and location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 96, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-96ae909f-1bee-4bee-a715-67674b7095bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Eurocup final match date and location'}, 'id': 'call_Gt5NplksLy9mAZXgoFxYdYxE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 96, 'output_tokens': 26, 'total_tokens': 122})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://en.wikipedia.org/wiki/UEFA_Euro_2024_Final", "content": "The UEFA Euro 2024 final was a football match that determined the 

### Adding memory

In [14]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [15]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "001"}}

In [16]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer Eurocup?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i0VE98BtGQRw8ZNB3AUvsNfy', 'function': {'arguments': '{"query":"2024 soccer Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 91, 'total_tokens': 114}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4cb77cd6-ee9d-4443-b995-f681792295c0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 soccer Eurocup winner'}, 'id': 'call_i0VE98BtGQRw8ZNB3AUvsNfy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 23, 'total_tokens': 114})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.nbcnewyork.com/news/sports/soccer/spain-england-euro-final-score-result/5595340/?os=os&ref=app", "content": "La Furia Roja on Sunday claimed the Euro 2

In [17]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of that winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='The top stars of the Spain team that won the 2024 soccer Eurocup included Mikel Oyarzabal, who scored a late winner in the final against England.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 354, 'total_tokens': 390}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2fa92989-91da-40c5-b5be-e19ad0298729-0', usage_metadata={'input_tokens': 354, 'output_tokens': 36, 'total_tokens': 390})]}}
----


In [18]:
config = {"configurable": {"thread_id": "002"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You were asking about the Real Madrid soccer team. Would you like me to provide you with more information about them?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 89, 'total_tokens': 113}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a43c93a1-49e6-440b-84e1-da49d8265aa9-0', usage_metadata={'input_tokens': 89, 'output_tokens': 24, 'total_tokens': 113})]}}
----
